In [1]:
# %pip install snowflake-connector-python
# %pip install pyarrow==8.0.0
# %pip install pandas
# %pip install confluent-kafka
# %pip install python-dotenv

In [2]:
import snowflake.connector
from snowflake_helpers import *
import re
import pandas as pd
from transformation_helpers import *

In [3]:
conn = connect_to_snowflake()
cs = conn.cursor()

Connected to Snowflake with user: admin


In [4]:
create_production_schema(cs)
use_production_schema(cs)

Schema: yusra_stories_production created.


In [5]:
df = cs.execute('SELECT * FROM yusra_stories_staging.logs').fetch_pandas_all()
df

,log_id,log
0,1,ride 1 --------- beginning of a new ride\n
1,2,ride 1 2022-10-06 15:49:21.453226 mendoza v9: ...
2,3,ride 1 2022-10-06 15:49:21.953902 mendoza v9: ...
3,4,ride 1 2022-10-06 15:49:22.454586 mendoza v9: ...
4,5,ride 1 2022-10-06 15:49:22.955260 mendoza v9: ...
...,...,...
41053,41054,ride 40 2022-10-06 21:32:15.156977 mendoza v9:...
41054,41055,ride 40 2022-10-06 21:32:15.657643 mendoza v9:...
41055,41056,ride 40 2022-10-06 21:32:16.158292 mendoza v9:...
41056,41057,ride 40 2022-10-06 21:32:16.658962 mendoza v9:...


In [6]:
formatted_df = get_joined_formatted_df(df)

at the start df is <class 'pandas.core.frame.DataFrame'>


/Users/harrymay/Desktop/coursework/deloton-group-project/transformation_helpers.py:89: FutureWarning: Comparison of NaT with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable.
  if birthday > today:
/Users/harrymay/Desktop/coursework/deloton-group-project/transformation_helpers.py:89: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if birthday > today:


In [7]:
formatted_df.head(3)

,log_id,log,ride_id,is_new_ride,is_info,is_system,time,user_id,name,gender,...,address,email_address,account_created,bike_serial,original_source,duration_secs,heart_rate,resistance,rpm,power
0,1,ride 1 --------- beginning of a new ride\n,1,True,False,False,NaT,<NA>,None,None,...,None,None,NaT,None,None,NaN,<NA>,NaN,NaN,NaN
1,2,ride 1 2022-10-06 15:49:21.453226 mendoza v9: ...,1,False,False,False,2022-10-06 15:49:21.453226,<NA>,None,None,...,None,None,NaT,None,None,NaN,<NA>,NaN,NaN,NaN
2,3,ride 1 2022-10-06 15:49:21.953902 mendoza v9: ...,1,False,False,True,2022-10-06 15:49:21.953902,4612,Mrs Bethany Welch,female,...,"Flat 3,Jeremy rapid,Evanshaven,S2 9DH",mrs_b@gmail.com,2022-02-27,SN0000,direct,NaN,<NA>,NaN,NaN,NaN


In [8]:
user_df = get_users_df(formatted_df)
user_df.head()

,name,gender,date_of_birth,age,height_cm,weight_kg,address,email_address,account_created,bike_serial,original_source
user_id,,,,,,,,,,,
4612,Mrs Bethany Welch,female,1968-09-20,54.0,188.0,60.0,"Flat 3,Jeremy rapid,Evanshaven,S2 9DH",mrs_b@gmail.com,2022-02-27,SN0000,direct
4614,Jill Gill,female,1996-11-11,25.0,180.0,61.0,"Studio 4,Carly valleys,West George,RH72 6JL",j_g86@gmail.com,2022-03-02,SN0000,google ads
4616,Mr Leon Taylor,male,1966-09-29,56.0,171.0,58.0,"Studio 82,Lloyd terrace,Allenport,PL6E 1YN",mr_leon@hotmail.com,2021-12-19,SN0000,direct
4617,Mitchell Tomlinson,male,1967-05-23,55.0,151.0,36.0,"520 Amy expressway,East Richardland,BH0V 6HL",mitchell_t@gmail.com,2022-03-15,SN0000,google ads
4619,Ms Clare Douglas,female,1981-08-28,41.0,170.0,57.0,"108 Davies square,Lake Stewartberg,W04 7NG",ms.c65@gmail.com,2021-12-18,SN0000,direct


## RIDE DF

In [9]:
staging_rides_df = get_staging_rides_df(formatted_df)

/Users/harrymay/Desktop/coursework/deloton-group-project/transformation_helpers.py:354: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staging_rides_df['total_duration'] = staging_rides_df.groupby(by=['ride_id'], dropna=False)['duration_secs'].transform('max')
/Users/harrymay/Desktop/coursework/deloton-group-project/transformation_helpers.py:355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staging_rides_df['total_duration'] = staging_rides_df['total_duration'].apply(lambda x: str(timedelta(seconds=x)))
/Us

In [10]:
rides_df = get_final_rides_df(staging_rides_df)
rides_df

,user_id,start_time,end_time,total_duration,max_heart_rate_bpm,min_heart_rate_bpm,avg_heart_rate_bpm,avg_resistance,avg_rpm,total_power_kilojoules
ride_id,,,,,,,,,,
1,4612,2022-10-06 15:49:21,2022-10-06 15:58:00,0:08:37,175,70,126,42,39,13.51
2,4614,2022-10-06 15:58:01,2022-10-06 16:06:42,0:08:40,139,62,91,41,52,13.10
3,4614,2022-10-06 16:06:44,2022-10-06 16:15:25,0:08:40,180,83,136,41,40,14.53
4,4614,2022-10-06 16:15:27,2022-10-06 16:24:07,0:08:39,134,65,100,41,45,11.08
5,4614,2022-10-06 16:24:08,2022-10-06 16:32:50,0:08:40,166,79,121,41,43,8.76
6,4614,2022-10-06 16:32:51,2022-10-06 16:40:55,0:08:03,165,82,133,42,42,13.56
7,4614,2022-10-06 16:40:57,2022-10-06 16:49:34,0:08:36,171,61,125,41,43,10.42
8,4614,2022-10-06 16:49:35,2022-10-06 16:58:17,0:08:40,144,78,104,41,43,12.93
9,4616,2022-10-06 16:58:18,2022-10-06 17:06:58,0:08:39,142,82,113,40,45,12.42


## For future table insert

In [11]:
# cs.execute(f"""
#         CREATE OR REPLACE TABLE df_test_table
#         ("log" STRING,
#         "is_new_ride" BOOLEAN,
#         "is_info" BOOLEAN,
#         "is_system" BOOLEAN,
#         "date" DATETIME,
#         "heart_rate" INTEGER,
#         "resistance" INTEGER,
#         "rpm" INTEGER,
#         "power" FLOAT,
#         "user_id" STRING,
#         "name" STRING,
#         "gender" STRING,
#         "address" STRING,
#         "date_of_birth" FLOAT,
#         "email_address" STRING,
#         "height_cm" INTEGER,
#         "weight_kg" INTEGER,
#         "account_create_date" FLOAT,
#         "bike_serial" STRING,
#         "original_source" STRING
#         )
# """)

In [12]:
# write_pandas(conn, df, 'DF_TEST_TABLE')

In [13]:
# cs.execute(f'SELECT * FROM DF_TEST_TABLE WHERE "is_system" = TRUE').fetch_pandas_all()